In [1]:
import random
import numpy as np

def normalize(arr):
    s = sum(arr)

    if s == 0:
        s = 1
        arr[0] = 1
    
    for i, val in enumerate(arr):
        arr[i] = val/s
    return arr


def generate(width, height):
    matrix = []

    for i in range(height):
        matrix.append([])

        for j in range(width):
            matrix[i].append(float(random.randint(0, 1000))/1000)

        matrix[i] = normalize(matrix[i])

        matrix[i] = [round(x, 3) for x in matrix[i]]
        
    return np.matrix(matrix)

generate(4,4)


print normalize([.444,.45, .34, .0])

[0.35980551053484605, 0.3646677471636953, 0.2755267423014587, 0.0]


In [2]:
#input:num characters
#output: num personality matricies, and closeness vectors

def initialize(num):
    out_matricies = []
    
    for i in range(num * 2):
        out_matricies.append(generate(4,4))
    
    closeness_vectors = []
    for x in range(num):
        close_vector = []
        for i in range(num):
            if i == x:
                close_vector.append(float(0))
            else:
                close_vector.append(float(random.randint(0,1000))/1000)
        closeness_vectors.append(normalize(close_vector))
    return out_matricies, closeness_vectors

initialize(2)

([matrix([[ 0.341,  0.384,  0.022,  0.253],
          [ 0.346,  0.087,  0.299,  0.267],
          [ 0.173,  0.266,  0.245,  0.316],
          [ 0.447,  0.252,  0.296,  0.004]]),
  matrix([[ 0.322,  0.422,  0.242,  0.014],
          [ 0.371,  0.305,  0.138,  0.186],
          [ 0.036,  0.375,  0.228,  0.361],
          [ 0.322,  0.116,  0.41 ,  0.152]]),
  matrix([[ 0.429,  0.145,  0.227,  0.2  ],
          [ 0.022,  0.417,  0.301,  0.26 ],
          [ 0.377,  0.208,  0.308,  0.107],
          [ 0.311,  0.122,  0.28 ,  0.287]]),
  matrix([[ 0.33 ,  0.104,  0.519,  0.047],
          [ 0.415,  0.39 ,  0.105,  0.09 ],
          [ 0.029,  0.428,  0.084,  0.46 ],
          [ 0.332,  0.081,  0.384,  0.204]])],
 [[0.0, 1.0], [1.0, 0.0]])

In [3]:
def traverse_row(matrix):
    rand  = float(random.randint(0,1000))/1000
    count = 0
    for i, elem in enumerate(matrix):
        if rand > count and rand < count + elem:
            return i
        count += elem
    return len(matrix) - 1

def translate(num):
    if num == 0:
        return "Happy"
    if num == 1:
        return "Sad"
    if num == 2:
        return "Angry"
    if num == 3:
        return "Fear"     
    
def translate_to_num(string):
    if num == "Happy":
        return 0
    if num == "Sad":
        return 1
    if num == "Angry":
        return 2
    if num == "Fear":
        return 3     

In [4]:
#This takes in one character, all the other characters in the frame, and all the matricies for 
#every character

def new_emotion_one_character(character_acting_id,    #id of acting character in frame
                              character_in_frame_ids, #id of other characters in frame
                              char_emotions,          #array of emotions of characters in frame
                              personality_matricies,  #array of personality matricies of characters in frame
                              impact_matricies,       #array of impact matricies of characters in frame
                              socialbility_params,    #array of socialbility of characters in frame
                              closeness_vectors):     #array of closeness vectors of characters in frame
    
    if len(character_in_frame_ids) == 0:
        P0   = personality_matricies[character_acting_id]
        emo0 = char_emotions[character_acting_id]
        
        if random.randint(0,1) == 1:
            position_and_direction = ['right', 'left']
        else:
            position_and_direction = ['left', 'right']
        
        return (traverse_row(P0.A[emo0]),position_and_direction)
    
    else:
        P0   = personality_matricies[character_acting_id]
        emo0 = char_emotions[character_acting_id]
        s0   = socialbility_params[character_acting_id]
        
        if random.randint(0,1) == 1:
            position_and_direction = ['right', 'left']
        else:
            position_and_direction = ['left', 'right']
        
        total_influence = np.matrix([[0.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, 0.0, 0.0],
                                     [0.0, 0.0, 0.0, 0.0]])
        
        for elem in character_in_frame_ids:
            M = impact_matricies[elem].A[char_emotions[elem]]
            v = closeness_vectors[character_acting_id][elem]
            M = np.repeat(M[np.newaxis,:], 4, 0)
            total_influence += v*M
    
        transition_matrix = (1-s0) * P0 + s0 *(total_influence)
    
        #return(traverse_row(transition_matrix.A[emo0]), position_and_direction)
        return(traverse_row(transition_matrix.A[emo0]), transition_matrix)
    

In [5]:
matrixes = initialize(3)
list1, list2 = matrixes
p0, im0, p1, im1, p2, im2 = list1
v0, v1, v2 = list2
soc0 = .3
soc1 = .4
soc2 = .5
emo0 = 0
emo1 = 0
emo2 = 0

print "Before:"
print "char0"
print "Personality Matrix 0:"
print p0
print "Impact Matrix 0:"
print im0
print "emotion:", translate(emo0)
print
print "char1"
print "Personality Matrix 1:"
print p1
print "Impact Matrix 1:"
print im1
print "emotion:", translate(emo1)
print
print "char2"
print "Personality Matrix 2:"
print p2
print "Impact Matrix 2:"
print im2
print "emotion:", translate(emo2)

Before:
char0
Personality Matrix 0:
[[ 0.095  0.283  0.358  0.264]
 [ 0.461  0.32   0.019  0.2  ]
 [ 0.374  0.06   0.387  0.179]
 [ 0.365  0.458  0.149  0.028]]
Impact Matrix 0:
[[ 0.351  0.238  0.214  0.198]
 [ 0.406  0.078  0.397  0.119]
 [ 0.233  0.599  0.147  0.022]
 [ 0.202  0.32   0.353  0.125]]
emotion: Happy

char1
Personality Matrix 1:
[[ 0.286  0.445  0.108  0.161]
 [ 0.087  0.295  0.158  0.46 ]
 [ 0.274  0.291  0.267  0.168]
 [ 0.125  0.213  0.407  0.255]]
Impact Matrix 1:
[[ 0.326  0.056  0.171  0.448]
 [ 0.137  0.404  0.071  0.388]
 [ 0.334  0.233  0.135  0.297]
 [ 0.405  0.287  0.1    0.208]]
emotion: Happy

char2
Personality Matrix 2:
[[ 0.127  0.384  0.204  0.285]
 [ 0.249  0.427  0.309  0.015]
 [ 0.204  0.362  0.224  0.211]
 [ 0.453  0.151  0.151  0.245]]
Impact Matrix 2:
[[ 0.428  0.112  0.062  0.398]
 [ 0.398  0.195  0.38   0.028]
 [ 0.437  0.33   0.122  0.111]
 [ 0.256  0.257  0.306  0.181]]
emotion: Happy


In [7]:

emo_0, direction_0 = new_emotion_one_character( 0, [1,2], [emo0, emo1, emo2], [p0, p1, p2],
                          [im0, im1, im2], [soc0, soc1, soc2], [v0, v1, v2])
emo_1, direction_1 = new_emotion_one_character( 1, [0,2], [emo0, emo1, emo2], [p0, p1, p2],
                          [im0, im1, im2], [soc0, soc1, soc2], [v0, v1, v2])
emo_2, direction_2 = new_emotion_one_character( 2, [0,1], [emo0, emo1, emo2], [p0, p1, p2],
                          [im0, im1, im2], [soc0, soc1, soc2], [v0, v1, v2])

print "After interaction:"
print "char0 emotion:", emo_0
print "char1 emotion:", translate(emo_1)
print "char2 emotion:", translate(emo_2)

print direction_0
print direction_1
print direction_2



After interaction:
char0 emotion: 2
char1 emotion: Fear
char2 emotion: Sad
[[ 0.18042239  0.22375151  0.28467118  0.31129687]
 [ 0.43662239  0.24965151  0.04737118  0.26649687]
 [ 0.37572239  0.06765151  0.30497118  0.25179687]
 [ 0.36942239  0.34625151  0.13837118  0.14609687]]
[[ 0.33590789  0.32307799  0.1032052   0.23789842]
 [ 0.21650789  0.23307799  0.1332052   0.41729842]
 [ 0.32870789  0.23067799  0.1986052   0.24209842]
 [ 0.23930789  0.18387799  0.2826052   0.29429842]]
[[ 0.2299264   0.24494416  0.1933934   0.33223604]
 [ 0.2909264   0.26644416  0.2458934   0.19723604]
 [ 0.2684264   0.23394416  0.2033934   0.29523604]
 [ 0.3929264   0.12844416  0.1668934   0.31223604]]
